In [ ]:
# [0] ]олучение картинок шахматной доски для калибровки искажений
# https://habr.com/ru/articles/341160/
import numpy as np
import cv2

img_counter =2
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) 

while(True): 
    ret, frame = cap.read()

    rows,cols,ch = frame.shape
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        img_name = "chessboard_{}.png".format(img_counter)
        img_counter=img_counter+1
        cv2.imwrite(img_name, frame)
        print(img_counter)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# [1] текущая рабочая версия  калибровки и корректировки перспективы
#  https://docs.opencv.org/3.1.0/dc/dbb/tutorial_py_calibration.html
# https://pavelk.ru/opencv-warpperspective-bez-obrezki-whole-image-i-razmer-rezultata-destination-result-size/

import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
chessboard_size = (9, 6) 

images = glob.glob('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\chessboard_8.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
  
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        src_points = np.float32([corners[0], corners[chessboard_size[0] - 1],
                            corners[-1], corners[-chessboard_size[0]]])
        width =  900 #int(np.linalg.norm(src_points[1] - src_points[0]))
        height =  600 #int(np.linalg.norm(src_points[2] - src_points[1]))
        #dst_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
        dst_points = np.float32([[width, height], [2*width, height], [2*width, 2*height], [width, 2*height]])

        print(width,height)

        M = cv2.getPerspectiveTransform(src_points, dst_points)
   
      

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners,ret)
        cv2.imshow('img',img)
        cv2.imwrite('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\c0.png',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

print(src_points)
print("-------------")
print(dst_points)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

img = cv2.imread('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\chessboard_7.png')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
cv2.imwrite('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\00.png',dst)

dst2=cv2.warpPerspective(dst,M,(2*1280,2*720),cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
cv2.imwrite('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\01.png',dst2)
 